In [24]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head() # 데이터 프레임의 첫번째 행부터 5대 추출 

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [25]:
# pandas unique : 명시된 컬럼에 중복 제거 후 항목 출력 
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [26]:
# Species : 타켓 데이터(정답 데이터), 그외 - 학습 데이터
import numpy as np
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height',  'Width']].to_numpy()
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [27]:
fish_target = fish['Species'].to_numpy() # 타켓 데이터(정답 데이터)

In [28]:
from sklearn.model_selection import train_test_split # 학습 세트, 테스트 세트 분리 

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target
)

In [29]:
# 데이터 전처리 - 표준 점수로 변환
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input) # 훈련 세트 훈련 데이터를 표준 점수로 변환 
test_scaled = ss.transform(test_input) # 테스트 세트 데이터를 표준 점수로 변환 
print(train_scaled[:5])

[[-0.74917842 -0.65434614 -0.70928686 -0.64685527 -0.61724995]
 [-0.64855124 -0.56062058 -0.51863871 -0.67446147 -0.34700189]
 [-0.66292655 -0.4950127  -0.56196783 -0.63813367 -0.4222287 ]
 [ 0.21396745  0.16106618  0.3479438   1.23058603  0.36233122]
 [-1.05479754 -1.56348401 -1.56720354 -1.60923664 -1.84016926]]


In [30]:
# K-최근접 이웃 분류 - 분류, 확률 예측 
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target) # 학습 

print(kn.score(train_scaled, train_target)) # 훈련 세트 정확도
print(kn.score(test_scaled, test_target)) # 테스트 세트 정확도 

0.8991596638655462
0.8


In [31]:
# 클래스 목록 (분류 항목) - 알파벳 순서대로 
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [32]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Perch' 'Bream' 'Bream' 'Smelt']


In [33]:
# 분류 기준 확률 kn.predict_proba(..)
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.    0.667 0.    0.333 0.    0.   ]
 [0.    0.    0.667 0.    0.333 0.    0.   ]
 [1.    0.    0.    0.    0.    0.    0.   ]
 [1.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.   ]]


In [34]:
distances, indexes = kn.kneighbors(test_scaled[2:3])
print(train_target[indexes]) # 확률로 보기에는 조금 어색하기 때문에 로지스틱 회귀를 사용!

[['Bream' 'Bream' 'Bream']]


In [35]:
# 로지스틱 회귀 - 이진 분류 (도미, 방어)
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]
print(target_bream_smelt[:5])

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [36]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt) # 훈련

LogisticRegression()

In [37]:
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [38]:
# 확률 predict_proba
print(np.round(lr.predict_proba(train_bream_smelt[:5]), decimals=3))

[[0.976 0.024]
 [0.03  0.97 ]
 [0.983 0.017]
 [0.993 0.007]
 [0.93  0.07 ]]


In [39]:
print(lr.classes_) # 0 : 음성 클래스, 1 : 양성 클래스

['Bream' 'Smelt']


In [40]:
# 가중치(기울기)와 절편 
print(lr.coef_, lr.intercept_)

[[-0.43062458 -0.57087436 -0.64692453 -0.97543711 -0.74372692]] [-1.8321658]


In [43]:
# 학습을 통해서 구한 가중치와 절편으로 방정식의 결과값을 구하는 함수 decision_function
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions) # 결과값을 0~1 사이로 변환 - 시그모이드 함수 

[-3.71118214  3.46676384 -4.05925792 -5.0175627  -2.58783602]


In [46]:
# 시그모이드 함수로 값을 0~1 사이로 변환
# 양성 클래스 함수 - 1에서 해당 값을 빼면 음성 클래스 함수 값
from scipy.special import expit # 시그모이드 함수 
print(np.round(expit(decisions), decimals=3))

[0.024 0.97  0.017 0.007 0.07 ]


In [49]:
# 로지스틱 회귀 - 다중 분류 / 확률구할 때 소프트맥스 함수 사용!
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target) # 학습 

print(lr.score(train_scaled, train_target)) # 학습 세트
print(lr.score(test_scaled, test_target)) # 테스트 세트

0.9411764705882353
0.9


In [50]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Perch' 'Bream' 'Bream' 'Smelt']


In [51]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [52]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.005 0.569 0.002 0.389 0.    0.035]
 [0.    0.    0.895 0.    0.099 0.    0.005]
 [0.998 0.    0.    0.    0.    0.    0.001]
 [0.976 0.001 0.    0.    0.    0.    0.023]
 [0.    0.002 0.046 0.    0.004 0.947 0.   ]]


In [53]:
print(lr.coef_, lr.intercept_)

[[-0.93216079 -1.1491407   1.64608277  7.4760879  -0.28108009]
 [-0.3946413  -2.0459944  -2.94775651  6.61067947 -2.57194665]
 [ 3.55678941  6.63502171 -8.96731895 -6.69228389  4.86645003]
 [ 0.01217059  3.6745293   4.01751804 -3.60880521 -1.53121656]
 [-3.18542133 -6.27594569  4.86118417 -0.46092892  2.66635323]
 [-0.7343207   1.12446214  1.34214    -4.29584391 -5.39834844]
 [ 1.67758412 -1.96293235  0.04815048  0.97109457  2.24978848]] [-0.3873433  -0.80512841  3.30605822 -0.01027819  1.52079531 -6.21656827
  2.59246464]


In [54]:
# decision_function
decisions = lr.decision_function(test_scaled[:5])
print(np.round(decisions, decimals=3))

[[ -3.895  -0.733   4.096  -1.487   3.717  -3.004   1.305]
 [ -5.557  -1.917   6.448  -1.59    4.25   -2.841   1.207]
 [ 13.191   5.408  -5.477  -2.099  -0.275 -17.191   6.443]
 [  9.98    3.144  -2.226  -1.815   0.729 -16.051   6.24 ]
 [-12.187   2.257   5.186  -4.152   2.696   8.207  -2.007]]


In [55]:
from scipy.special import softmax

proba = softmax(decisions, axis=1) # 행별로 구한다! (axis=1)
print(np.round(proba, decimals=3))

[[0.    0.005 0.569 0.002 0.389 0.    0.035]
 [0.    0.    0.895 0.    0.099 0.    0.005]
 [0.998 0.    0.    0.    0.    0.    0.001]
 [0.976 0.001 0.    0.    0.    0.    0.023]
 [0.    0.002 0.046 0.    0.004 0.947 0.   ]]
